## 1. Facial Detection and Image Capturing for Dataset


In [ ]:
print("*******************************************************\n")     
print("Welcome to the Facial Access Control Center\n")
print("This program will register you on the database and be used to grant you access to restricted areas\n")
print("Your facial records are kept securely on this database\nData collected contains no identifying information and isn't shared with third parties\n")
print("*******************************************************\n")

consent_answer = input("Press q and enter if you do not consent and wish to quit\nPress any other key and enter to continue ===>  ")

import sys
if consent_answer == ("q") or consent_answer == ("Q"):
    print("Thanks for your response, good bye\n")
    quit()
else:
    print("Welcome to the program")
    
import cv2
import numpy as np
count = 0
import os

#Setting up dnn
modelFile = "dnn_config/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn_config/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

#Setting up the webcam
cap = cv2.VideoCapture(0)
#Font for the caption
font = cv2.FONT_HERSHEY_SIMPLEX 
# For each person, enter a face id
face_id = input('\n enter user id end press <return> ==>  ')

#Creates a working directory if it doesn't exist
os.makedirs("headshots", exist_ok=True)

# Gets current working directory
path = "headshots"

# Joins the folder that we wanted to create
folder_name = (face_id)
path = os.path.join(path, folder_name) 

# Creates the folder, and checks if it is created or not.
os.makedirs(path, exist_ok=True)



print("\n [INFO] Initializing face capture. Look the camera and wait ...")
while(True):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    
    if ret == True:
        img = cv2.resize(img, None, fx=0.8, fy=0.8)
        height, width = img.shape[:2]
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                                        1.0, (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces3 = net.forward()
        
        for i in range(faces3.shape[2]):
            confidence = faces3[0, 0, i, 2]
            if confidence > 0.5:
                box = faces3[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
                count += 1
            
        # Save the captured image into the datasets folder
            cv2.imwrite(os.path.join(path, str(face_id) + '.' +  
                    str(count) + ".jpg"), gray[y:y1,x:x1])
            
        k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 60: # Take 60 face sample and stop video
            break
        cv2.putText(img, 'Facial Detector', (30, 30), font, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        #######################

        cv2.imshow("dnn", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

        

           
cap.release()
cv2.destroyAllWindows()

## 2. Image Augmentation and Model Training

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dense, Conv2D, MaxPool2D , Flatten

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
#Evaluation imports
import sklearn.metrics as metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
#augmentation of training images, best used when the dataset is small
train_datagen = ImageDataGenerator(
    rescale=1/255.,              # normalize pixel values between 0-1
    brightness_range=[0.1, 0.7], # specify the range in which to decrease/increase brightness
    width_shift_range=0.5,       # shift the width of the image 50%
    rotation_range=90,           # random rotation by 90 degrees
    horizontal_flip=True,        # 180 degree flip horizontally
    vertical_flip=True,          # 180 degree flip vertically
    validation_split=0.2        # 20% of the data will be used for validation at end of each epoch
)

#This is used to count how many classes of images you have
train_generator = train_datagen.flow_from_directory(
'./Headshots',
target_size=(224,224),
color_mode='rgb',
batch_size=32,
class_mode='categorical',
shuffle=False, subset = 'training', seed=42)

validation_generator = train_datagen.flow_from_directory(
'./Headshots',
target_size=(224,224),
color_mode='rgb',
batch_size=32,
class_mode='categorical',
shuffle=False, subset = 'validation', seed=42)

In [ ]:
train_generator.class_indices.values()
# dict_values([0, 1, 2])
NO_CLASSES = len(train_generator.class_indices.values())
NO_CLASSES

In [ ]:
train_generator.class_indices.keys()


In [ ]:
train_generator.class_indices.values()

In [ ]:
from keras_vggface.vggface import VGGFace
pbase_model = VGGFace(include_top=False,
weights= 'vggface', #VGGFace is lightweight and thus can be transmitted across the network
input_shape=(224, 224, 3))
pbase_model.trainable = False
pbase_model.summary()
print(len(pbase_model.layers))
# 19 layers after excluding the last few layers


In [ ]:
#from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1024, activation='relu')
dense_layer_2 = layers.Dense(1024, activation='relu')
dense_layer_3 = layers.Dense(512, activation='relu')
prediction_layer = layers.Dense(NO_CLASSES, activation='softmax')


#Coupling both the top and the bottom layers
pmodel = models.Sequential([
    pbase_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    prediction_layer
])



In [ ]:

pmodel.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='accuracy', mode='max', patience=7)

history = pmodel.fit(train_generator, epochs=50, batch_size=32, verbose = 1,  validation_data=validation_generator, callbacks=[es])
#validation_steps=10
#

In [ ]:
train_loss, train_acc = pmodel.evaluate(train_generator, verbose=2)
print('\n Training accuracy:', train_acc)
print('\n Training loss:', train_loss)
test_loss, test_acc = pmodel.evaluate(validation_generator, verbose=2)
print('\n Test accuracy:', test_acc)
print('\n Test loss:', test_loss)

## Measurements

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
# Plotting historic Test and Training Accuracy
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(1,38)
plt.subplots(figsize = (10,10))
plt.plot(epochs, acc_train, 'g', label='Historic Training Accuracy')
plt.plot(epochs, acc_val, 'r', label='Historic Validation Accuracy')
# Add a horizontal line (Test Accuracy, Valication Accuracy)
plt.axhline(test_acc, linestyle = "--", color = 'y' ,label='Test accuracy')
plt.axhline(train_acc, linestyle = "--", label='Train accuracy')
plt.title('Training accuracy CNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("plot_acc3.png")
plt.show()

In [ ]:
# Plotting Test and Training loss
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,38)
plt.subplots(figsize = (10,10))
plt.plot(epochs, loss_train, 'g', label='Historic Training Loss')
plt.plot(epochs, loss_val, 'r', label='Historic Validation Loss')
# Add a horizontal line (Test Loss, training Loss)
plt.axhline(test_loss, linestyle = "--", color = 'y', label='Test Loss')
plt.axhline(train_loss, linestyle = "--", label='Training Loss')
plt.title('Training Loss CNN')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("plot_loss3.png")
plt.show()

In [ ]:
#Getting Prediction Labels
y_pred = pmodel.predict(validation_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
#Classification Report

class_report = metrics.classification_report(validation_generator.classes, y_pred_labels, target_names=train_generator.class_indices.keys(),zero_division=1,  output_dict=False)
print(class_report)    
class_report1 = classification_report(validation_generator.classes, y_pred_labels, target_names=train_generator.class_indices.keys(),zero_division=1, output_dict=True)


In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(15,10)) 
plt.rcParams['font.size'] = '20'
sns.heatmap(pd.DataFrame(class_report1).iloc[:-1, :].T, annot=True,)
plt.savefig('class_report3.jpg')

In [ ]:
#Confusion Matrix
conf_matrix = (confusion_matrix(validation_generator.classes, y_pred_labels))
print(conf_matrix)

In [ ]:

fig, ax = plt.subplots(figsize=(15,12)) 
ww = sns.heatmap(conf_matrix, annot=True, xticklabels= (train_generator.class_indices.keys()), yticklabels= (train_generator.class_indices.keys()));
ww.set_xlabel('True Class', fontsize=25)
ww.set_ylabel('Predicted Class', fontsize=25)

# Export plot
plt.savefig('conf_matrix3.jpg')

In [ ]:
import pickle

class_dictionary = train_generator.class_indices
class_dictionary = {
    value:key for key, value in class_dictionary.items()
}
print(class_dictionary)



In [ ]:
# save the class dictionary to pickle
face_label_filename = 'face-labels.pickle'
with open(face_label_filename, 'wb') as f: pickle.dump(class_dictionary, f)


In [ ]:
# creates a HDF5 file
pmodel.save(
    'transfer_learning_facial_recognition' +
    '_face_cnn_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# deletes the existing model
del pmodel

# returns a compiled model identical to the previous one
pmodel = load_model(
    'transfer_learning_facial_recognition' +
    '_face_cnn_model.h5')


# Running Facial Recognition


In [1]:
from PIL import Image
import numpy as np
import cv2
import pickle
from tensorflow.keras.models import load_model

count = 0
import os

# for face detection
modelFile = "dnn_config/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn_config/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)


# resolution of the webcam
screen_width = 1280       # try 640 if code fails
screen_height = 720

# size of the image to predict
image_width = 224
image_height = 224

# load the trained model
model = load_model('transfer_learning_facial_recognition_face_cnn_model.h5')


# the labels for the trained model
with open("face-labels.pickle", 'rb') as f:
    og_labels = pickle.load(f)
    labels = {key:value for key,value in og_labels.items()}
    print(labels)

# default webcam
stream = cv2.VideoCapture(0)
stream.set(3, 1280) # set video width
stream.set(4, 720) # set video height
# Define min window size to be recognized as a face
minW = 0.1*stream.get(3)
minH = 0.1*stream.get(4)

while(True):
    ret, img = stream.read()
    (grabbed, frame) = stream.read()
    frame = cv2.flip(frame, 1)
    
    if grabbed == True:
        frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
        height, width = frame.shape[:2]
        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300,300)),
                                        1.0, (300,300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces3 = net.forward()
        
        for i in range(faces3.shape[2]):
            confidence = faces3[0, 0, i, 2]
            if confidence > 0.5:
                box = faces3[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)
                count += 1


        # predict the image
        frame1 = cv2.resize(frame, (224, 224))
        frame1 = frame1.reshape(1,224,224,3)
        predicted_prob = model.predict(frame1)

        # Display the label
        font = cv2.FONT_HERSHEY_SIMPLEX
        name = labels[predicted_prob[0].argmax()]
        color = (255, 0, 255)
        stroke = 2
        cv2.putText(frame, f'({name})', (x,y-8),
            font, 1, color, stroke, cv2.LINE_AA)
        

        # Show the frame
        cv2.imshow("Image", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):    # Press q to break out of the loop
            break      

# Cleanup
stream.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


{0: 'Gabby', 1: 'Gloria', 2: 'Guy', 3: 'Lolomari', 4: 'Nike', 5: 'man', 6: 'ola', 7: 'quo'}
1/1 [==============================] - 0s 251ms/step


-1

In [ ]:
frame.shape

In [ ]:
height


In [ ]:
predicted_prob